# Check the skipped instances from MA

In [15]:
import json
# missing_qn = []
missing_id = []
# Replace 'your_file.json' with the path to your JSON file
with open('/home/yl3427/cylab/SOAP_MA/Output/TNM/brca_final_with_baseline_500.json', 'r') as file:
    data = json.load(file)
print("Total number of items:", len(data))

for idx, item in enumerate(data):
    if len(item.keys()) != 9: # soap, tnm은 9개, medicalqa는 11개
        print("Error in line", idx)
        print(item['File ID'])
        missing_id.append(item['File ID'])
#     try:
#         question = item['Question']
#     except:
#         print("Error in line", idx)
#         print(item['qn_num'])
#         missing_qn.append(item['qn_num'])

Total number of items: 400


In [18]:
# print(missing_qn)
print(missing_id)

[]


In [3]:
len(missing_id)

3

In [ ]:
for idx, item in enumerate(data):
    if item['Fi

351

# Check the skipped instances from Baseline

In [16]:
import json
missing_item = []
# Replace 'your_file.json' with the path to your JSON file
with open('/home/yl3427/cylab/SOAP_MA/Output/TNM/brca_only_with_baseline.json', 'r') as file:
    data = json.load(file)

for idx, item in enumerate(data):
    if item["BaselineChoice"] == "Unknown":
        print("Error in line", idx)
        print(item['filename'])
        missing_item.append(item['filename'])

In [7]:
error_cnt = 0
for idx, item in enumerate(data):
    label = f"T{item['Answer']+1}"
    baseline_pred = item['BaselineChoice']
    if label != baseline_pred:
        print(label, baseline_pred)
        error_cnt += 1
print(error_cnt)

T2 T1
T2 T1
T3 T1
T3 T2
T2 T1
T3 T2
T2 T4
T1 T2
T3 T2
T3 T2
T1 T2
T1 T2
T1 T2
T2 T4
T1 T4
T4 T2
T3 T2
T4 T3
T2 T4
T3 T2
T1 T2
T2 T1
T2 T1
T1 T2
T2 T4
T3 T2
T1 T2
T3 T4
T3 T2
T1 T2
T4 T2
T3 T1
T2 T4
T1 T2
T1 T2
T2 T3
T3 T4
T1 T2
T1 T2
T1 T2
T3 T2
T2 T1
T3 T4
T3 T4
T3 T2
T3 T1
T1 T2
T3 T2
T4 T2
T4 T2
T3 T2
T1 T2
T3 T1
T3 T1
T1 T2
T2 T1
T3 T1
T1 T2
T3 T1
T3 T2
T1 T2
T3 T4
T1 T2
T1 T2
T1 T2
T2 T1
T1 T2
T1 T2
T3 T2
T3 T2
T1 T2
T2 T3
T1 T2
T3 T2
T1 T2
T2 T4
T4 T2
T4 T1
T2 T3
T3 T4
T1 T3
T1 T2
T3 T4
T1 T2
T1 T2
T2 T4
T3 T2
T2 T4
T1 T2
T1 T2
T2 T4
T4 T3
T2 T1
T2 T4
T2 T1
T2 T1
T2 T4
T2 T4
T1 T2
T3 T4
T2 T3
T1 T2
T1 T2
T1 T2
T2 T1
T2 T1
T1 T2
T1 T4
T4 T2
T4 T2
T3 T2
T4 T2
T2 T1
T3 T2
114


In [2]:
len(missing_item)

0

In [7]:
missing_qn = []

In [8]:
input_json_path = "sample_data.json"
f"{input_json_path.split('.')[0]}_with_baseline.json"


'sample_data_with_baseline.json'

In [4]:
import pandas as pd
df = pd.read_csv("/home/yl3427/cylab/llm_reasoning/reasoning/data/step3_ALL.csv")
len(df)

137

In [4]:
input_json_path.split(".")

['sample_data', 'json']

In [16]:
import json
import copy
import os

def deep_merge(dict1, dict2):
    """
    Recursively merge dict2 into dict1.
    - If a key exists in both dicts and both values are themselves dicts, 
      recurse into those.
    - Otherwise, dict2’s value overwrites dict1’s value.
    """
    for key, value in dict2.items():
        if key not in dict1:
            dict1[key] = value
        else:
            if isinstance(dict1[key], dict) and isinstance(value, dict):
                deep_merge(dict1[key], value)
            else:
                # Overwrite dict1's value with dict2's value
                dict1[key] = value
    return dict1

def merge_json_files(file1, file2, output_file, identifier='qn_num'):
    """
    Merges two JSON files (each containing a list of dicts) 
    on a specified 'identifier' key. 
    If the key is missing from a particular dict, it will be skipped.
    """
    # 1) Load both JSON files
    with open(file1, "r", encoding="utf-8") as f:
        data1 = json.load(f)  # List of dicts
    with open(file2, "r", encoding="utf-8") as f:
        data2 = json.load(f)  # List of dicts

    # 2) Convert the first list into a dictionary keyed by 'identifier'
    merged_dict = {}
    for item in data1:
        if identifier in item:
            key_value = item[identifier]
            merged_dict[key_value] = copy.deepcopy(item)

    # 3) For each item in data2, either add or merge
    for item in data2:
        if identifier not in item:
            # Skip if 'identifier' doesn't exist in this item
            continue
        key_value = item[identifier]
        if key_value not in merged_dict:
            merged_dict[key_value] = copy.deepcopy(item)
        else:
            # Deep-merge fields from item into existing
            deep_merge(merged_dict[key_value], item)

    # 4) Convert the merged dictionary back to a list
    final_list = list(merged_dict.values())

    print(f"Merged {len(data1)} items from file1 and {len(data2)} items from file2.")
    print(f"Final list contains {len(final_list)} items.")

    # 5) Write to the output file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(final_list, f, indent=2, ensure_ascii=False)

if __name__ == "__main__":
    # Example usage:
    file1 = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_missing_final.json"
    file2 = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final.json"
    output_file = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final.json"

    # Example identifier: 'File ID' or 'qn_num'
    # If you need to merge on 'File ID', just pass identifier='File ID'.
    merge_json_files(file1, file2, output_file, identifier='File ID')
    print(f"Merged file saved as: {output_file}")


Merged 1 items from file1 and 351 items from file2.
Final list contains 351 items.
Merged file saved as: /home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final.json


In [17]:
import pandas as pd
# df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/Input/SOAP_5_problems.csv")
df = pd.read_csv("/home/yl3427/cylab/SOAP_MA/Input/step3_ALL.csv")
len(df)

137

In [13]:
json_path = "/home/yl3427/cylab/SOAP_MA/Output/SOAP/sepsis_final.json"
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)
len(data)

351

# Evaluation

In [5]:
with open('/home/yl3427/cylab/SOAP_MA/Output/SOAP/sepsis_final_with_baseline.json', 'r') as file:
    data = json.load(file)
tp = 0
fp = 0
tn = 0
fn = 0
_ = 0

for item_dict in data:
    pred = item_dict['BaselineChoice']
    if 'sepsis' in item_dict['Answer']:
        label = "Yes"
    elif 'septic' in item_dict['Answer']:
        label = "Not sure"
    else:
        label = "No"

    if pred == "Yes" and label == "Yes":
        tp += 1
    elif pred == "Yes" and label == "No":
        fp += 1
    elif pred == "No" and label == "No":
        tn += 1
    elif pred == "No" and label == "Yes":
        fn += 1
    else:
        _ += 1

print("True positives:", tp)
print("True negatives:", tn)
print("False positives:", fp)
print("False negatives:", fn)
print("Other:", _)

# Calculate precision, recall, and F1 score
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

True positives: 80
True negatives: 131
False positives: 93
False negatives: 24
Other: 23
Precision: 0.4624277456647399
Recall: 0.7692307692307693
F1 Score: 0.5776173285198556


In [5]:
import json

def evaluate_SOAP_predictions(
    file_path,
    terms_lst: list,
    aggregator_prediction_key="final_choice",
    baseline_prediction_key="BaselineChoice",
):
   
    def get_label(answer, terms_lst):
        for term in terms_lst:
            if term.lower() in answer.lower():
                return "Yes"
        return "No"

    def compute_confusion_matrix(data_list, prediction_key, is_aggregator=True):
      
        tp = fp = tn = fn = other = 0

        for item in data_list:
            if is_aggregator:
                # aggregator prediction is nested: item[aggregator_key][aggregator_prediction_key]
                pred = item["Aggregator"][prediction_key]
            else:
                # baseline prediction is top-level: item[baseline_prediction_key]
                pred = item[prediction_key]

            label = get_label(item["Answer"], terms_lst)

            if pred == "Yes" and label == "Yes":
                tp += 1
            elif pred == "Yes" and label == "No":
                fp += 1
            elif pred == "No" and label == "No":
                tn += 1
            elif pred == "No" and label == "Yes":
                fn += 1
            else:
                # e.g., pred is "Yes" but label is "Not sure", or vice versa
                other += 1

        return tp, tn, fp, fn, other

    def calculate_metrics(tp, tn, fp, fn):
        precision = tp / (tp + fp) if (tp + fp) else 0.0
        recall = tp / (tp + fn) if (tp + fn) else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0.0

        return {
            "Precision": precision,
            "Recall": recall,
            "F1 Score": f1
        }

    def print_results(title, tp, tn, fp, fn, other, metrics):
        """
        Nicely prints out the confusion matrix and performance metrics.
        """
        print(f"\n{title}")
        print("-" * len(title))
        print(f"{'True Positives (TP)':25} : {tp}")
        print(f"{'True Negatives (TN)':25} : {tn}")
        print(f"{'False Positives (FP)':25} : {fp}")
        print(f"{'False Negatives (FN)':25} : {fn}")
        print(f"{'Other':25} : {other}")

        print()
        for metric_name, value in metrics.items():
            print(f"{metric_name:25} : {value:.3f}")

    # ----------------------------
    # 2. Load data from JSON file
    # ----------------------------
    with open(file_path, "r") as file:
        data = json.load(file)

    # ---------------------------------------------------
    # 3. Compute confusion matrix & metrics for aggregator
    # ---------------------------------------------------
    agg_tp, agg_tn, agg_fp, agg_fn, agg_other = compute_confusion_matrix(
        data, aggregator_prediction_key, is_aggregator=True
    )
    agg_metrics = calculate_metrics(agg_tp, agg_tn, agg_fp, agg_fn)

    # -------------------------------------------------
    # 4. Compute confusion matrix & metrics for baseline
    # -------------------------------------------------
    base_tp, base_tn, base_fp, base_fn, base_other = compute_confusion_matrix(
        data, baseline_prediction_key, is_aggregator=False
    )
    base_metrics = calculate_metrics(base_tp, base_tn, base_fp, base_fn)

    print_results("Aggregator Agent Performance", agg_tp, agg_tn, agg_fp, agg_fn, agg_other, agg_metrics)
    print_results("Baseline Performance", base_tp, base_tn, base_fp, base_fn, base_other, base_metrics)

In [6]:
mi = ["myocardial infarction", "elevation mi", "non-stemi", " NSTEMI", " stemi"]
chf = ["congestive heart failure", " chf", "HFrEF", "HFpEF"]
pulmonary_embolism = ["pulmonary embolism"]
pulmonary_hypertension = ["pulmonary hypertension", "pulmonary htn"]
sepsis = ["sepsis", "septic shock"]
urosepsis = ["urosepsis"]
meningitis = ["meningitis"]
aki = ["acute kidney injury", " aki", "acute renal failure", " arf"] # -> Acute tubular necrosis (ATN)인가 아닌가
atn = ["acute tubular necrosis", " atn"]
pancreatitis = ["pancreatitis"]
gi_bleed = ["gastrointestinal bleed", "gi bleed"]
hepatitis = ["hepatitis", " hep"]
cholangitis = ["cholangitis"]
asp_pneumonia = ["aspiration pneumonia"]
evaluate_SOAP_predictions("/home/yl3427/cylab/SOAP_MA/Output/SOAP/chf_final_with_baseline.json", chf)


Aggregator Agent Performance
----------------------------
True Positives (TP)       : 39
True Negatives (TN)       : 217
False Positives (FP)      : 42
False Negatives (FN)      : 53
Other                     : 0

Precision                 : 0.481
Recall                    : 0.424
F1 Score                  : 0.451

Baseline Performance
--------------------
True Positives (TP)       : 37
True Negatives (TN)       : 209
False Positives (FP)      : 50
False Negatives (FN)      : 55
Other                     : 0

Precision                 : 0.425
Recall                    : 0.402
F1 Score                  : 0.413


### Medical QA

In [ ]:
# 시발 존나 짜증나네. 오지선다 아닌것도 있네 시발. 아오 좆같아시발.
with open("/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_final_with_baseline.json", 'r') as file:
    data = json.load(file)
# for k, v in data[0].items():
    # print(k, v)

import re

for item in data:
    text = item["Question"].split("\n")[-1]
    matches = re.findall(r'\b[A-Z](?=[.:])', text)
    print(matches)


In [75]:
import json
from collections import Counter
import numpy as np

# If you have scikit-learn installed:
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    classification_report,
)

def evaluate_multiclass_predictions(
    data,
    aggregator_key="Aggregator",
    aggregator_prediction_key="final_choice",
    baseline_prediction_key="BaselineChoice",
    classes=["A","B","C","D","E"]
):
    
    # ------------------------------------
    # 2. Collect gold labels and predictions
    # ------------------------------------
    gold_labels_agg = []
    pred_labels_agg = []

    gold_labels_base = []
    pred_labels_base = []

    for item in data:
        # The gold label is presumably in item["Answer"], e.g. 'A', 'B', etc.
        gold = item["Answer"]
        
        # Aggregator's prediction
        # e.g. item["Aggregator"]["final_choice"] is 'A', 'B', 'C', etc.
        agg_pred = item[aggregator_key][aggregator_prediction_key]

        # Baseline's prediction
        base_pred = item[baseline_prediction_key]

        gold_labels_agg.append(gold)
        pred_labels_agg.append(agg_pred)

        gold_labels_base.append(gold)
        pred_labels_base.append(base_pred)

    # ------------------------------------------------
    # 3. Build confusion matrices & compute stats
    #    (Using scikit-learn for convenience)
    # ------------------------------------------------

    print("== Aggregator Results ==")
    cm_agg = confusion_matrix(gold_labels_agg, pred_labels_agg, labels=classes)
    print("Confusion Matrix (rows=True label, cols=Predicted):")
    print(cm_agg, "\n")

    accuracy_agg = accuracy_score(gold_labels_agg, pred_labels_agg)
    print(f"Accuracy: {accuracy_agg:.3f}\n")

    # classification_report prints precision, recall, f1 for each class,
    # plus macro/micro/weighted averages
    report_agg = classification_report(
        gold_labels_agg, 
        pred_labels_agg,
        labels=classes,
        zero_division=0  # to handle any class with no predictions
    )
    print(report_agg)

    print("\n== Baseline Results ==")
    cm_base = confusion_matrix(gold_labels_base, pred_labels_base, labels=classes)
    print("Confusion Matrix (rows=True label, cols=Predicted):")
    print(cm_base, "\n")

    accuracy_base = accuracy_score(gold_labels_base, pred_labels_base)
    print(f"Accuracy: {accuracy_base:.3f}\n")

    report_base = classification_report(
        gold_labels_base, 
        pred_labels_base,
        labels=classes,
        zero_division=0
    )
    print(report_base)



In [76]:
import re
filtered_data = []
total_data_len = 0
for i in range(1, 4):
    with open("/home/yl3427/cylab/SOAP_MA/Output/MedicalQA/step1_final_with_baseline.json", 'r') as file:
        data = json.load(file)
    total_data_len += len(data)

    for item in data:
        text = item["Question"].split("\n")[-1]
        matches = re.findall(r'\b[A-Z](?=[.:])', text)
        if set(matches) == {'A', 'B', 'C', 'D', 'E'}:
            filtered_data.append(item)

evaluate_multiclass_predictions(filtered_data)

== Aggregator Results ==
Confusion Matrix (rows=True label, cols=Predicted):
[[63  0  0  0  3]
 [ 0 48  3  0  0]
 [ 6  0 36  0  0]
 [ 0  0  6 63  0]
 [ 0  3  3  0 33]] 

Accuracy: 0.910

              precision    recall  f1-score   support

           A       0.91      0.95      0.93        66
           B       0.94      0.94      0.94        51
           C       0.75      0.86      0.80        42
           D       1.00      0.91      0.95        69
           E       0.92      0.85      0.88        39

    accuracy                           0.91       267
   macro avg       0.90      0.90      0.90       267
weighted avg       0.92      0.91      0.91       267


== Baseline Results ==
Confusion Matrix (rows=True label, cols=Predicted):
[[60  0  3  0  3]
 [ 0 48  3  0  0]
 [ 3  3 36  0  0]
 [ 0  3  6 57  3]
 [ 0  3  3  0 33]] 

Accuracy: 0.876

              precision    recall  f1-score   support

           A       0.95      0.91      0.93        66
           B       0.84      

In [74]:
len(filtered_data), total_data_len

(267, 357)